# Zakharov function
---
Description:

- Optimization (min)
- Single-objective
- Constraints (no)
---

Minimize the equation given by:

- $f\left(\mathbf{x}\right) = \sum_{i=1}^{d}x_i^2 + \left(\sum_{i=1}^{d}0.5ix_i\right)^2 + \left(\sum_{i=1}^{d}0.5ix_i\right)^4$

where: $-10 \le x_i \le 10$.

Global Minimum:

- $f\left(\mathbf{\hat{x}}\right) = 0$, at $\mathbf{\hat{x}} = [0, 0, ..., 0]$

### First we import python libraries and set up the directory of our code.

In [7]:
import os, sys
import numpy as np
from math import fsum, isclose

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Here we import all our custom GA code.

In [8]:
# Import main classes.
from pygenalgo.genome.gene import Gene
from pygenalgo.genome.chromosome import Chromosome
from pygenalgo.engines.standard_ga import StandardGA
from pygenalgo.utils.utilities import cost_function

# Import Selection Operator(s).
from pygenalgo.operators.selection.linear_rank_selector import LinearRankSelector

# Import Crossover Operator(s).
from pygenalgo.operators.crossover.meta_crossover import MetaCrossover

# Import Mutation Operator(s).
from pygenalgo.operators.mutation.meta_mutator import MetaMutator

### Define the objective function, which plays also the role of the 'fitness' function.

In addition, we define the 'boundary_x' which takes the role of the 'random()' method of the Genes. Every 
time we want to 'mutate' a gene this function will be called that returns 'valid', but random values for 
the gene.

In [9]:
# Objective function.
@cost_function(minimize=True)
def fun_Zakharov(individual: Chromosome):
    
    # Extract gene values from the chromosome.
    x = individual.values()
    
    # Compute the repeated sum.
    sum_xi = 0.5*fsum([i*xi for i, xi in enumerate(x, start=1)])
    
    # Compute the final value.
    f_value = fsum([xi**2 for xi in x]) + sum_xi**2 + sum_xi**4

    # Condition for termination.
    solution_found = isclose(f_value, 0.0, rel_tol=1.0e-6)
    
    # Return the solution tuple.
    return f_value, solution_found
# _end_def_

Here we set the GA parameters, such as number of genes, number of chromosomes, etc.

In [10]:
# Define the number of optimization variables.
M = 8

# Define the number of chromosomes.
N = 100

# Random number generator.
rng = np.random.default_rng()

# Random boundary function:
boundary_x = lambda: rng.uniform(-10.0, 10.0)

# Draw random samples for the initial points.
x_init = rng.uniform(-10.0, +10.0, size=(N, M))

# Initial population.
population = [Chromosome([Gene(x_init[i, j], boundary_x)
                          for j in range(M)], np.nan, True)
              for i in range(N)]

# Create the StandardGA object that will carry on the optimization.
test_GA = StandardGA(initial_pop=population,
                     fit_func=fun_Zakharov,
                     select_op=LinearRankSelector(),
                     mutate_op=MetaMutator(),
                     crossx_op=MetaCrossover())

### Optimization process.

Here we call the GA object (either directly, or through the method run()). We set a number of parameter, such as the maximum iterations (i.e. epochs), tolerance for the fitness convergences, etc.

In [11]:
test_GA(epochs=1000, elitism=True, f_tol=1.0e-8, adapt_probs=True)

Initial Avg. Fitness = -8173978.4999
Final   Avg. Fitness = -41572.9701
Elapsed time: 20.212 seconds.


In [12]:
# Extract the optimal solution from the GA.
optimal_solution = test_GA.best_chromosome()

# Extract the fitness value from the optimal solution.
optimal_fit = -optimal_solution.fitness

# Display the (final) optimal value.
print(f"Minimum Found: {optimal_fit:.6f}\n")

# Display each gene value separately.
for i, xi in enumerate(optimal_solution.values(), start=1):
    print(f"x{i} = {xi:>10.6f}")
# _end_for_

Minimum Found: 0.001108

x1 =  -0.008405
x2 =  -0.008405
x3 =  -0.008405
x4 =   0.017944
x5 =  -0.008405
x6 =  -0.008405
x7 =   0.017944
x8 =  -0.008405


Global Minimum:
$f\left(\mathbf{\hat{x}}\right) = 0$, at $\mathbf{\hat{x}} = [0, 0, ..., 0]$

### End of file